import a pandas e al quick draw dataset.

In [ ]:
import os
!mkdir quickdraw
!mkdir bitmaps strokes sframes
os.chdir('bitmaps/')
classes = [    'airplane',    'angel',    'apple',    'axe',    'banana',    'baseball',    'bee',    'bicycle',    'bird',    'book',    'bowtie',    'bridge',    'butterfly',    'cactus',    'camera',    'car',    'carrot',    'castle',    'cat',    'chair',    'church',    'circle',    'clock',    'cloud',    'coffee cup',    'compass',    'cookie',    'crab',    'dog',    'dolphin',    'donut',    'door',    'dragon',    'duck',    'elephant',    'fire hydrant',    'fish',    'flower',    'flying saucer']

# Bitmap data (around 90 MB each)
for elem in classes:
  !curl https://storage.googleapis.com/quickdraw_dataset/full/numpy_bitmap/{elem}.npy > {elem}.npy

mkdir: cannot create directory ‘quickdraw’: File exists
mkdir: cannot create directory ‘bitmaps’: File exists
mkdir: cannot create directory ‘strokes’: File exists
mkdir: cannot create directory ‘sframes’: File exists
  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100  113M  100  113M    0     0   198M      0 --:--:-- --:--:-- --:--:--  197M
  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100  111M  100  111M    0     0   115M      0 --:--:-- --:--:-- --:--:--  114M
  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100  108M  100  108M    0     0   176M      0 --:--:-- --:--:-- --:--:--  175M
  % Total    % Received % Xferd  Average Speed   Time    Time     Time 

In [ ]:
#os.chdir('./quickdraw')
!ls

airplane.npy  bicycle.npy    camera.npy  circle.npy   dog.npy	    fire
angel.npy     bird.npy	     car.npy	 clock.npy    dolphin.npy   fish.npy
apple.npy     book.npy	     carrot.npy  cloud.npy    donut.npy     flower.npy
axe.npy       bowtie.npy     castle.npy  coffee       door.npy	    flying
banana.npy    bridge.npy     cat.npy	 compass.npy  dragon.npy
baseball.npy  butterfly.npy  chair.npy	 cookie.npy   duck.npy
bee.npy       cactus.npy     church.npy  crab.npy     elephant.npy


In [ ]:
classes = [    'airplane',    'angel',    'apple',    'axe',    'banana',    'baseball',    'bee',    'bicycle',    'bird',    'book',    'bowtie',    'bridge',    'butterfly',    'cactus',    'camera',    'car',    'carrot',    'castle',    'cat',    'chair',    'church',    'circle',    'clock',    'cloud',    'compass',    'cookie',    'crab',    'dog',    'dolphin',    'donut',    'door',    'dragon',    'duck',    'elephant',    'fish',    'flower']


In [ ]:
import os
os.chdir('bitmaps/')

In [ ]:
!pip install keras-tuner

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 176.1/176.1 kB 7.3 MB/s eta 0:00:00


In [ ]:
import tensorflow as tf
import numpy as np
import os
from tensorflow.keras.applications.resnet50 import ResNet50
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.optimizers import SGD
import matplotlib.pyplot as plt
from tensorflow.keras.preprocessing.image import ImageDataGenerator

random_state = np.random.RandomState(100)

# Change if applicable
quickdraw_dir = './'
#bitmaps_dir = os.path.join(quickdraw_dir, 'bitmaps')
bitmaps_dir=quickdraw_dir
npy_ext = '.npy'
num_examples_per_class = 6000
num_classes = len(classes)

def build_bitmap_dataset():
    bitmaps_list, labels_list = [], []
    for class_name in classes:
        class_data = np.load(os.path.join(bitmaps_dir, class_name + npy_ext))
        random_state.shuffle(class_data)
        class_data_selected = class_data[:num_examples_per_class]
        class_data_selected = class_data_selected.reshape(
            class_data_selected.shape[0], 28, 28, 1)
        bitmaps_list.append(class_data_selected)
        labels_list.append(np.full((num_examples_per_class,), class_name))

    bitmaps_array = np.concatenate(bitmaps_list, axis=0)
    labels_array = np.concatenate(labels_list, axis=0)
    return bitmaps_array, labels_array

bitmaps, labels = build_bitmap_dataset()
num_samples = bitmaps.shape[0]
image_shape = (32,32,3)
input_shape = (None,) + image_shape
rgb_bitmaps = np.repeat(bitmaps, 3, axis=-1)

class_indices = {cls: i for i, cls in enumerate(classes)}
labels = np.array([class_indices[label] for label in labels])

# Preprocessing
rgb_bitmaps = bitmaps.astype("float32") / 255.
bitmaps_resized = tf.image.resize(bitmaps, [32, 32])
labels = tf.keras.utils.to_categorical(labels, num_classes)


# Train/Val/Test split
train_ratio, val_ratio = 0.6, 0.2
train_idx = int(num_samples * train_ratio)
val_idx = train_idx + int(num_samples * val_ratio)
train_bitmaps, train_labels = bitmaps[:train_idx], labels[:train_idx]
val_bitmaps, val_labels = bitmaps[train_idx:val_idx], labels[train_idx:val_idx]
test_bitmaps, test_labels = bitmaps[val_idx:], labels[val_idx:]
#train
train_bitmaps= tf.image.resize(np.repeat(train_bitmaps,3,axis=-1),(32,32))
#test
test_bitmaps= tf.image.resize(np.repeat(test_bitmaps,3,axis=-1),(32,32))
#val
val_bitmaps= tf.image.resize(np.repeat(val_bitmaps,3,axis=-1),(32,32))


Random Search

In [ ]:
import tensorflow as tf
from tensorflow import keras
import keras_tuner as kt
import numpy as np
import os

# Definisci il modello
def model_builder(hp):
    # Carica ResNet50
    base_model = tf.keras.applications.ResNet50(input_shape=(32, 32, 3),
                                                include_top=False,
                                                weights='imagenet')
    base_model.trainable = False

    # Aggiungi il nuovo layer di output
    model = keras.Sequential()
    model.add(base_model)
    model.add(tf.keras.layers.Flatten())
    model.add(tf.keras.layers.Dense(512, activation=hp.Choice('dense_activation', ['relu', 'tanh', 'sigmoid'])))
    model.add(tf.keras.layers.Dropout(hp.Float('dropout', 0, 0.5, step=0.1)))
    model.add(tf.keras.layers.Dense(num_classes, activation='softmax'))

    # Scegli l'ottimizzatore e la learning rate
    optimizer = hp.Choice('optimizer', ['adam', 'sgd'])
    lr = hp.Float('learning_rate', 1e-5, 1e-2, sampling='log')

    if optimizer == 'adam':
        opt = tf.keras.optimizers.Adam(learning_rate=lr)
    else:
        opt = tf.keras.optimizers.SGD(learning_rate=lr)

    # Compila il modello
    model.compile(optimizer=opt,
                  loss='categorical_crossentropy',
                  metrics=['accuracy'])

    return model

# Crea l'oggetto tuner
tuner = kt.RandomSearch(model_builder,
                        objective='val_accuracy',
                        max_trials=10,
                        directory='random_search',
                        project_name='quickdraw_classification')

# Esegui la ricerca
batch_size,epochs=2000,3
tuner.search(train_bitmaps, train_labels, epochs=epochs, batch_size=batch_size, validation_data=(val_bitmaps, val_labels))

# Ottieni i migliori parametri
best_hps = tuner.get_best_hyperparameters(num_trials=1)[0]

print(f"Learning rate: {best_hps.get('learning_rate'):.4f}")
print(f"Dense activation: {best_hps.get('dense_activation')}")
print(f"Dropout: {best_hps.get('dropout'):.2f}")
print(f"Optimizer: {best_hps.get('optimizer')}")


Search: Running Trial #1

Value             |Best Value So Far |Hyperparameter
relu              |relu              |dense_activation
0                 |0                 |dropout
adam              |adam              |optimizer
0.00017384        |0.00017384        |learning_rate

Epoch 1/3
 4/65 [>.............................] - ETA: 8:50 - loss: 5.4371 - accuracy: 0.0551


KeyboardInterrupt



BITMAP DATA INTO EXAMPLES

In [ ]:
# Define the model
resnet = ResNet50(weights='imagenet', include_top=False, input_shape=image_shape)

# Congela tutti i layer pre-esistenti
layer_attivi=2
for layer in resnet.layers[:-layer_attivi]:
    layer.trainable = False


# Aggiungi un nuovo layer di output per le tue classi
x = tf.keras.layers.Flatten()(resnet.output)
x=tf.keras.layers.Dense(512, activation="tanh")(x)
x = tf.keras.layers.Dropout(0.10)(x)  # aggiungi il layer di dropout
x = tf.keras.layers.Dense(num_classes, activation='softmax')(x)

# Crea il nuovo modello
model = tf.keras.models.Model(inputs=resnet.input, outputs=x)
"""model = tf.keras.models.Sequential([
    tf.keras.layers.Conv2D(32, (3, 3), activation='relu', input_shape=image_shape),
    tf.keras.layers.MaxPooling2D(pool_size=(2, 2)),
    tf.keras.layers.Conv2D(64, (3, 3), activation='relu'),
    tf.keras.layers.MaxPooling2D(pool_size=(2, 2)),
    tf.keras.layers.Flatten(),
    tf.keras.layers.Dense(128, activation='relu'),
    tf.keras.layers.Dense(num_classes, activation='softmax')
])"""
#inserire SGD eventualmente
#opt=SGD(learning_rate=0.001,momentum=0.9)
opt = Adam(learning_rate=0.0003, beta_1=0.95, beta_2=0.999, epsilon=1e-03, amsgrad=False)
model.compile(loss='categorical_crossentropy',
              optimizer=opt,
              metrics=['accuracy'])
print(model.summary())
#data augm
datagen = ImageDataGenerator(
    rotation_range=8,  # Rotazione casuale di 10 gradi
    width_shift_range=0.1,  # Traslazione orizzontale casuale del 10% della larghezza dell'immagine
    height_shift_range=0.1,  # Traslazione verticale casuale del 10% dell'altezza dell'immagine
    zoom_range=0.05,  # Zoom casuale del 10%
    horizontal_flip=True)  # Flip orizzontale casuale

# Fit del data generator sui dati di training
datagen.fit(train_bitmaps)
batch_size, epochs = 104, 3
# Esegui il training del modello utilizzando il data generator
history = model.fit(datagen.flow(train_bitmaps, train_labels, batch_size=batch_size),
                    epochs=epochs,
                    verbose=1,
                    validation_data=(val_bitmaps, val_labels))
# Train the model
"""batch_size, epochs = 32, 5
history = model.fit(train_bitmaps, train_labels,
                    batch_size=batch_size,
                    epochs=epochs,
                    verbose=1,
                    validation_data=(val_bitmaps, val_labels))"""

# Plot the training and validation loss per epoch
plt.plot(history.history['accuracy'], label='Training accuracy')
plt.plot(history.history['val_accuracy'], label='Validation accuracy')
plt.xlabel('Epoch')
plt.ylabel('Accuracy')
plt.legend()

plt.show()

# Evaluate the model on the test set
test_loss, test_acc = model.evaluate(test_bitmaps, test_labels, verbose=0)
print(f"Test accuracy: {test_acc:.4f}")


Model: "model"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_2 (InputLayer)           [(None, 32, 32, 3)]  0           []                               
                                                                                                  
 conv1_pad (ZeroPadding2D)      (None, 38, 38, 3)    0           ['input_2[0][0]']                
                                                                                                  
 conv1_conv (Conv2D)            (None, 16, 16, 64)   9472        ['conv1_pad[0][0]']              
                                                                                                  
 conv1_bn (BatchNormalization)  (None, 16, 16, 64)   256         ['conv1_conv[0][0]']             
                                                                                              

KeyboardInterrupt: ignored

In [ ]:
import random
import matplotlib.pyplot as plt

# Scegli un'immagine a caso dal test set
index = random.randint(0, len(test_bitmaps)-1)
image = test_bitmaps[index]
label = test_labels[index]
class_index = np.argmax(label) 
print(class_index)

# Visualizza l'immagine
plt.imshow(image)
plt.title(f"Label: {label}")
plt.show()
